# Packages

In [2]:
import ee
import geemap
import geopandas as gpd

import json

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Open Nighttime Lights'

# Convert local shapefile to EE object

Sometimes it’s necessary to work with vector data not available in the GEE data catalog. This can often be the case with custom created geometries.

To do that, we’ll use the Python json and geopandas module, which is a comprehensive library for geospatial analytics and data processing in Python, and create a helper function.

In [4]:
ROOT_DIR = os.path.dirname(os.path.abspath())
ROOT_DIR

TypeError: abspath() missing 1 required positional argument: 'path'

In [9]:
# You may need to change this
sydney_path = 'C:/Users/gilramolete/OneDrive - UNIONBANK of the Philippines/Documents 1/Open Nighttime Lights/files/city_of_sydney_shapefile/sydney.shp'

# Read shapefile via geopandas, and initiate the crs
sydney_gdf = gpd.read_file(sydney_path, crs = 'EPSG:4326')

# Define a helper function to put the geodataframe in the right format for constructing an ee object
def shp_to_ee_fmt(geodf):
    data = json.loads(geodf.to_json())
    return data['features'][0]['geometry']['coordinates']

# Create the ee object
sydney = ee.Geometry.MultiPolygon(shp_to_ee_fmt(sydney_gdf))
sydney

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.MultiPolygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              [
                151.231110942926,
                -33.8708414667619
              ],
              [
                151.231110932167,
                -33.8708417786118
              ],
              [
                151.231129337281,
                -33.8708549026688
              ],
              [
                151.231225733,
                -33.8709027878742
              ],
              [
                151.231308269488,
                -33.8709309431124
              ],
              [
                151.231393556826,
                -33.8709650406197
              ],
              [
                151.231446215493,
                -33.8709923205043
              ],
              [
                151.231470587612,
                -33.87101253764
              ],
              [
                151.231488751988,
                -33.8710303662095
              ],
              [
                151.231503153541,
                -33.8710516936475
              ],
              [
                151.231524640628,
                -33.8710885643034
              ],
              [
                151.231531483516,
                -33.8711003058816
              ],
              [
                151.231543551075,
                -33.8711260905129
              ],
              [
                151.231489957344,
                -33.8711506819599
              ],
              [
                151.231489558996,
                -33.871150864707
              ],
              [
                151.231490072633,
                -33.8711508576519
              ],
              [
                151.231508878784,
                -33.8711510455387
              ],
              [
                151.231515358302,
                -33.8711662527257
              ],
              [
                151.231531795692,
                -33.8711746783321
              ],
              [
                151.23155106044,
                -33.8711913098379
              ],
              [
                151.231548524961,
                -33.8712044849383
              ],
              [
                151.231576078598,
                -33.8712281635379
              ],
              [
                151.231568915517,
                -33.8712355064465
              ],
              [
                151.231581073859,
                -33.8712638614541
              ],
              [
                151.231595578273,
                -33.871272973491
              ],
              [
                151.231615772236,
                -33.871344498641
              ],
              [
                151.231623911128,
                -33.8713532100097
              ],
              [
                151.231634110408,
                -33.8713641259643
              ],
              [
                151.231628506949,
                -33.8713981586225
              ],
              [
                151.231618958175,
                -33.8714413387828
              ],
              [
                151.231614090434,
                -33.8716129579133
              ],
              [
                151.231604310576,
                -33.8716270389897
              ],
              [
                151.231590544628,
                -33.87165638353
              ],
              [
                151.23157803323,
                -33.8716933000092
              ],
              [
                151.231555031613,
                -33.8717018547629
              ],
              [
                151.231484293807,
                -33.8718706274927
              ],
              [
                151.231462759994,
                -33.8718747901485
              ],
              [
                151.231394693029,
                -33.87197083

# Clip Sydney shapefile to VIIRS-DNB 2018 Composite

Get VIIRS-DNB and create annual composite. We'll use the `avg_rad` band.

In [10]:
viirs2018 = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')\
            .filterDate('2018-01-01', '2018-12-31')\
            .select('avg_rad')\
            .median()

# Clip to our Sydney shapefile
sydney_viirs = viirs2018.clip(sydney)

# Initialize the map, and stretch the min/max values to stretch so we can see their dynamics of a bright city center
sydMap = geemap.Map()
sydMap.add_basemap('SATELLITE')
sydMap.centerObject(sydney, zoom = 13)
sydMap.addLayer(sydney_viirs, {'min': 1, 'max': 100}, 'VIIRS 2018', opacity = 0.75)
sydMap.addLayerControl()
sydMap

Map(center=[-33.889462622418314, 151.20275194560503], controls=(WidgetControl(options=['position', 'transparen…

With the VIIRS resolution, even for such a small area, you can see differentiation for brightness around the dense areas, such as the downtown neighborhood by the harbor and the stadiums to the Southeast of that.

It might also be useful to note, that the native spatial resolution of VIIRS-DNB is approximately 500m. While that’s higher thand DMPS-OLS and small enough to be useful for this Area of Interest, you can see the pixel size would limit meaningful analysis or visualizing for an AOI a bit smaller than this.

# Export clipped image to GDrive

You may want to save a clipped file for use in other analyses. With GEE, your only immediate option is to export your data to a location in your Google Drive account (and from there you can download locally if you’d like).

You can export to GDrive useing the `ee.batch.Export.image.toDrive` function to create and start a task.

As with running the `reduceRegion` function we saw in other exercises you’ll also pass a scale parameter and set maxPixels to above the default.

Other parameters include:
- explicitly set the output format to GeoTIFF.
- set folder to “tmp” a folder we’ve created. If you dont set this, the file will be saved to your root.
- set the description to the name of the outgoing file

In [ ]:
# task = ee.batch.Export.image.toDrive(image = sydney_viirs,
#                                      scale = 30,
#                                      fileFormat = 'GeoTIFF',
#                                      description = 'Sydney_VIIRS_2018',
#                                      folder = 'tmp',
#                                      maxPixels = 1e9)
# task.start()